##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

# 装载nbfunc中的hub涉及的所有ipynb函数
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb

initGit()

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [

]

tools_to_install = [
    'p7zip-full',
    'iputils-ping',
]

def extract_library_name(library):
    """从库名中提取出包的名称"""
    return library.split("==")[0]

def get_installed_libraries():
    """获取已安装的库列表"""
    output = subprocess.check_output(["pip", "freeze"])
    installed_libraries = output.decode().split("\n")
    return set([extract_library_name(library) for library in installed_libraries])

def install_library(library):
    """安装指定的库"""
    !pip install $library --user

# 获取已安装的库列表
installed_libraries = get_installed_libraries()

# 找出未安装的库并安装
libraries_to_install_filtered = [library for library in libraries_to_install if extract_library_name(library) not in installed_libraries]
if libraries_to_install_filtered:
    print(f"以下库将被安装：{libraries_to_install_filtered}")
    for library in libraries_to_install_filtered:
        install_library(library)
else:
    print("所有需要安装的库都已经安装。")

# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

In [ ]:
import os
import subprocess

def compress_pylibs(arch_dir, target_file):
    # 检查压缩文件夹路径是否存在
    if not os.path.exists(arch_dir):
        raise ValueError(f'{arch_dir}文件夹不存在！')
    
    # 检查输出路径是否存在，若不存在则创建该路径
    if not os.path.exists(os.path.dirname(target_file)):
        os.makedirs(os.path.dirname(target_file))
    
    # 使用7z最高压缩级别的方法，压缩文件夹
    if not os.path.exists(target_file):
        !7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on {target_file} {arch_dir}
    
    # 获取文件大小并计算压缩比
    file_size = os.path.getsize(target_file)
    !du -sh {target_file}
    arch_size = sum(os.path.getsize(os.path.join(root, file)) for root, dirs, files in os.walk(arch_dir) for file in files)
    compressed_ratio = file_size / arch_size * 100
    print(f'压缩完成！\n压缩比: {compressed_ratio:.2f}%')

compress_pylibs('/root/.pylibs', f'{root_dir}/pylibs.7z')
compress_pylibs(f'/{root_dir}/py3.10.6', f'{root_dir}/py3.10.6.7z')